In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).parent.absolute()))

In [3]:
from train.generate_bc_data import record_bc_data

record_bc_data(
  save_dir=Path('D:/bc-train-data-test'),
  total_sample_count=10000,
  max_steps_per_rollout=600,
  skip_frames=2,
  camera_dims=(128, 128)
)

Allocating 3.93GB in `D:\bc-train-data-test`...


Attempting rollout 0.:  74%|███████▎  | 441/600 [01:18<00:28,  5.62it/s]


 - Rollout succeeded. Saved 148 samples.


Attempting rollout 1.:  65%|██████▌   | 391/600 [00:49<00:26,  7.89it/s]


 - Rollout succeeded. Saved 131 samples.


...

 - Rollout succeeded. Saved 141 samples.


Attempting rollout 70.: 100%|██████████| 600/600 [01:08<00:00,  8.75it/s]


 - Rollout failed. Discarded 200 samples.


Attempting rollout 70.:  85%|████████▍ | 508/600 [01:18<00:14,  6.43it/s]


 - Rollout succeeded. Saved 170 samples.
Finished generating 10000 samples in D:\bc-train-data-test.
